In [1]:
import numpy as np
import json
import glob


#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# #spacy
import spacy
from nltk.corpus import stopwords

#visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
#prepping data
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

    return data


def write_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [3]:
stopwords = stopwords.words("english")


In [4]:
# loads descriptions of episodes
episodes = load_data("data/plots.json")

# episode titles in the form of "Season;Episode"
names = [str(episode["Season"]) + ";" + str(episode["No. inseason"])
         for episode in episodes]

# maps episode titles to their index in the names array
names_map = {}
for index in range(len(names)):
    names_map[names[index]] = index

# episode summaries
plots = [episode["plot"] for episode in episodes]

# episode summaries in the form of "Season;Episode\nSummary"
summaries = [str(episode["Season"]) + ";" + str(episode["No. inseason"]
                                                ) + "\n" + str(episode["plot"]) for episode in episodes]

# loads episode scripts
episodes_data = {}
data = load_data("data/scripts.json")

for line in data:
    episode_title = line["episode_name"]
    script_line = line["dialogue"]

    if episode_title not in episodes_data:
        episodes_data[episode_title] = ""

    episodes_data[episode_title] += script_line + " "

episodes_array = []
# Iterate through the episodes_data dictionary and extract the script lines
for lines in episodes_data.values():
    # Combine all script lines into a single string for each episode
    episode_string = "".join(lines)
    episodes_array.append(episode_string)

# clean up the episode scripts and combine them with the episode summaries
for i in range(len(episodes_array)):
    episodes_array[i] = episodes_array[i].replace("\\n", " ")
    episodes_array[i] = episodes_array[i].replace("\\", "")
    episodes_array[i] = episodes_array[i].replace("  ", " ")
    plots[i] = plots[i] + " " + episodes_array[i]
    
    

In [7]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    """https://spacy.io/api/annotation"""
    nlp = spacy.load("/Users/anishpalakurthi/opt/anaconda3/lib/python3.8/site-packages/en_core_web_sm/en_core_web_sm-3.6.0", disable=["parser", "ner"])
    texts_out = []
    for sent in texts:
        #contains metadata about the word
        doc = nlp(sent)
        #lemmatizes each word by appending allowed tokens from the doc object's metadata
        texts_out.append(
            " ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags]))
    return texts_out

print("path successfully hit")
lemmatized_texts = lemmatization(plots)
print(lemmatized_texts[0][0:90])

path successfully hit
unsuccessful visit high iq sperm bank return home find aspire actress penny new neighbor h


In [11]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
        
    return final

#preprocess our lemmatized keywords
data_words = gen_words(lemmatized_texts)

print(data_words[0][0:10])

['unsuccessful', 'visit', 'high', 'iq', 'sperm', 'bank', 'return', 'home', 'find', 'aspire']


In [14]:
id2word = corpora.Dictionary(data_words)

corpus = [id2word.doc2bow(text) for text in data_words]

for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

#[index, frequency]
print(corpus[0][0:10])


[(0, 3), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]
actress


In [15]:
#generate LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word = id2word, num_topics=30, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds = "mmds", R = 30)
vis

/Users/anishpalakurthi/Library/Python/3.10/lib/python/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.326829  0.162741       1        1  44.191397
8      0.342688  0.088406       2        1  37.895640
3      0.335009  0.127450       3        1   8.769889
9      0.267790  0.255100       4        1   2.613170
1      0.395321 -0.005282       5        1   2.082300
14     0.338406 -0.086850       6        1   1.890315
25     0.112888  0.252812       7        1   0.700640
21     0.214423 -0.255915       8        1   0.547641
19    -0.015391 -0.328940       9        1   0.379844
20    -0.231592  0.199110      10        1   0.336666
24    -0.133786  0.262124      11        1   0.245834
10    -0.044794 -0.208892      12        1   0.174820
16    -0.223509 -0.137966      13        1   0.158066
15    -0.099075 -0.019053      14        1   0.000823
2     -0.099075 -0.019053      15        1   0.000821
6     -0.099075 -0.019053      16        1   0.000812
18    -0.099075 -0.019053      17        1   0.000810
4     -0.099075 -0.019053      18        1   0.000810
11    -0.099075 -0.019053      19        1   0.000810
23    -0.099075 -0.019053      20        1   0.000809
22    -0.099075 -0.019053      21        1   0.000809
0     -0.099075 -0.019053      22        1   0.000809
12    -0.099075 -0.019053      23        1   0.000809
7     -0.099075 -0.019053      24        1   0.000809
13    -0.099075 -0.019053      25        1   0.000809
5     -0.099075 -0.019053      26        1   0.000809
28    -0.099075 -0.019053      27        1   0.000808
29    -0.099075 -0.019053      28        1   0.000808
27    -0.099075 -0.019053      29        1   0.000807
26    -0.099075 -0.019053      30        1   0.000806, topic_info=           Term         Freq         Total Category  logprob  loglift
220          go  11486.00000  11486.000000  Default  30.0000  30.0000
554        time   2894.00000   2894.000000  Default  29.0000  29.0000
438          re   8923.00000   8923.000000  Default  28.0000  28.0000
288        just   7230.00000   7230.000000  Default  27.0000  27.0000
217         get   8950.00000   8950.000000  Default  26.0000  26.0000
..          ...          ...           ...      ...      ...      ...
613        work      0.00032   2407.632815  Topic30  -9.5248  -4.1052
0          able      0.00032    202.165714  Topic30  -9.5248  -1.6279
1        access      0.00032     65.115662  Topic30  -9.5248  -0.4950
2    accomplish      0.00032     46.170762  Topic30  -9.5248  -0.1512
3        across      0.00032      4.189996  Topic30  -9.5248   2.2485

[2028 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3557     12  0.651213     aaaa
3387     13  0.662269  aaaaaaa
0         1  0.375929     able
0         2  0.321518     able
0         3  0.192911     able
...     ...       ...      ...
620       5  0.031592     year
620       6  0.016725     year
620       7  0.017654     year
3555     13  0.823643    yield
3681     12  0.785084       zz

[2423 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 9, 4, 10, 2, 15, 26, 22, 20, 21, 25, 11, 17, 16, 3, 7, 19, 5, 12, 24, 23, 1, 13, 8, 14, 6, 29, 30, 28, 27])